# Sales Modeling - Data Preparation
This script takes a raw CSV file generated by Salesforce and converts it into a CSV file that is ready to use for training machine learning models.

3/4/24 - This script is not finished.

#### Import dependencies

In [ ]:
# Define some exclusions for PEP8 that don't apply when the Jupyter Notebook
#   is exported to .py file
# pylint: disable=pointless-statement
# pylint: disable=fixme
# pylint: disable=expression-not-assigned
# pylint: disable=missing-module-docstring
# pylint: disable=invalid-name

import os
import json
# import sys
# import re
from math import isnan
from collections import Counter

import pandas as pd
# from pandas._libs.tslibs.parsing import DateParseError
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

sns.set_theme()

### Load the data into a Pandas dataframe
Define the path to the dataset file
Define the name of the label column

In [ ]:
rootdir = os.getcwd()
infile = os.path.join(rootdir, 'data', 'sales_data.csv')
df = pd.read_csv(infile)
# df.dtypes

#### Customized variables for this dataset

In [ ]:
# can be either "train" or "predict"
DATA_PREP_MODE = 'train'

LABEL_COLUMN_NAME = "Won"
# NORMALIZE_METHOD = "min_max"

FEATURE_VALID_RANGES = {
    'Age': {'min': 20, 'max': 720},
    'Annual Recurring Revenue (ARR)': {'min': 2000, 'max': 2000000},
    'quarter_closed': {'min': 1, 'max': 4},
    'quarter_created': {'min': 1, 'max': 4},
}

FEATURE_PROPER_DATATYPES = {"Annual Recurring Revenue (ARR)": "int64",
               "Age": "int64",
               'partner_involved': "bool"
               }

FEATURES_TO_OHE = ['Team Territory Group', 'Opportunity Owner',
                         'Industry', 'primary_product']

# def get_stat(col_name, stat_name):
#     """docstring TBD"""
    # return df.describe(include="all").loc[stat_name].loc[col_name]

# Finding the percentiles:
# def find_nearest_index(array, value):
#     """docstring TBD"""
#     array = np.asarray(array)
#     idx = (np.abs(array - value)).argmin()
#     return idx

# validate configuration above.
assert DATA_PREP_MODE in ['train', 'predict'], \
    "DATA_PREP_MODE must be either train or predict. Exiting"
    
if DATA_PREP_MODE == 'train':
    FEATURE_PROPER_DATATYPES[LABEL_COLUMN_NAME] = 'bool'
    # print(FEATURE_PROPER_DATATYPES)

# df.head(10)

## Correcting/converting the feature data types

#### Extracting the Salesforce JSON fields into individual fields
Salesforce has a strict limitation on the number of columns it can export in a report. One way around this is to combine several fields into a single one using JSON.

In [ ]:
# Establishing invalid default values to create the features
df['quarter_created'] = 0
df['quarter_closed'] = 0
df['primary_product'] = None
df['partner_involved'] = None

# convert JSON to new columns
for index_iter in df.index:
    fields_as_json_str = df['array_of_sfdc_formulas'][index_iter]
    fields_as_dict = json.loads(fields_as_json_str)
    for colname, value in fields_as_dict.items():
        df.loc[df.index == index_iter, colname] = value


for colname, newdatatype in FEATURE_PROPER_DATATYPES.items():
    if DATA_PREP_MODE == "train" or (DATA_PREP_MODE == "predict" and colname != LABEL_COLUMN_NAME):
        df[colname] = df[colname].astype(newdatatype)

# df.dtypes

## Drop the unused features

In [ ]:
df.drop(
    columns=[
        "Annual Recurring Revenue (ARR) Currency",
        "Opportunity ID",     # leave this is for easy re-joining the data
        "array_of_sfdc_formulas"
    ],
    inplace=True,
)

# df.dtypes

## Correcting salesforce user error and other non-use cases

Removes the following:
* opps with $0 or less ARR - these are refunds, free upgrades, etc.
* opps with an age of less than 14 days - these were likely duplicates or multiple opps made for quoting purposes
* 

In [ ]:
for colname, ranges_dict in FEATURE_VALID_RANGES.items():
    min_value = ranges_dict['min']
    max_value = ranges_dict['max']
    indexes_to_drop = df[ (df[colname] < min_value) | (df[colname] > max_value) ].index
    df.drop(indexes_to_drop, inplace=True)

## Winsorizing numerical outliers
Description TBD

In [ ]:
# # Winsorize the top 1% and bottom 1%
# percentile = 0.01

# for iter_column_name in df.select_dtypes(include=np.number).columns.tolist():
#     new_column_name = iter_column_name + "_winsorized"

#     winsorized_data = stats.mstats.winsorize(
#         df[iter_column_name], limits=[percentile, percentile], inplace=False
#     )

#     if (winsorized_data == df[iter_column_name]).all():
#         print(
#             f"Winsorization on column {iter_column_name} had no effect. Not changing this column."
#         )
#         continue

#     df[new_column_name] = winsorized_data
#     df.drop(columns=iter_column_name, inplace=True)
#     print(
#         f"Winsorized column {iter_column_name} to {new_column_name} and removed original column."
#     )

## Replacing missing numerical values w/ their mean
Description TBD

## Normalizing numerical ranges
Description TBD

In [ ]:
# def normalize(df_local, column_name, normalize_method_name):
#     """docstring TBD"""
#     df_temp = df_local.copy()
#     new_columnname = column_name + "_normalized"

#     if normalize_method_name == "absolute_range":
#         df_temp[new_columnname] = (
#             df_temp[column_name] / df_temp[column_name].abs().max()
#         )

#     elif normalize_method_name == "min_max":
#         # rescales a features to be in the range [0,1]
#         df_temp[new_columnname] = (
#             df_temp[column_name] - df_temp[column_name].min()
#         ) / (df_temp[column_name].max() - df_temp[column_name].min())

#     elif normalize_method_name == "z_score":
#         df_temp[new_columnname] = (
#             df_temp[column_name] - df_temp[column_name].mean()
#         ) / df_temp[column_name].std()

#     else:
#         raise NameError("Unrecogized normalization method")

#     df_temp.drop(columns=column_name, inplace=True)
#     print(
#         f"Normalized column {column_name} into {new_columnname} using {normalize_method_name}. Removed original."
#     )
#     return df_temp


# # iterate through the list of current numeric columns
# for iter_column_name in df.select_dtypes(include=np.number).columns.tolist():
#     df = normalize(df, iter_column_name, NORMALIZE_METHOD)

# One hot encoding strings
sales rep names, industries, products

In [ ]:
for iter_column_name in FEATURES_TO_OHE:
    # define a new column names, it automatically adds a _ to the end of the prefix
    new_column_prefix = iter_column_name

    # create a one-hot encoded version in a new dataframe
    temp_df = pd.get_dummies(df[iter_column_name], prefix=new_column_prefix)

    # merge the new dataframe into the existing one
    df = df.join(temp_df)

    # remove the original column now that it has been encoded 
    # into the existing dataframe
    df.drop(columns=iter_column_name, inplace=True)
    
    print(f'One-hot encoded: {iter_column_name} into {new_column_prefix}*')
    
df.dtypes

# Re-order the columns
Sort the column names alphabetically, but make sure the 'label' column is always last.

In [ ]:
# alphabetically sort the column names, but leave the label as the last column
# since the label will be dropped for predictive datasets
column_order = sorted(df.columns)

if DATA_PREP_MODE == "train":
    column_order.remove(LABEL_COLUMN_NAME)
    column_order.append(LABEL_COLUMN_NAME)

df = df.reindex(column_order, axis=1)

# Final tests

In [ ]:
# check for missing values
# check for any remaining strings
df.describe(include="all")

In [ ]:
# show the final datatypes before exporting to CSV
df.dtypes

In [ ]:
df.head()

In [ ]:
#TODO: check the % of labels that are True vs False, ensure balance

# Basic data validation before modeling

Check ranges, values, datatypes, missing values, 

In [ ]:
for colname, ranges_dict in FEATURE_VALID_RANGES.items():
    min_value = ranges_dict['min']
    max_value = ranges_dict['max']
    for index, row in df.iterrows():
        val = df[colname][index]
        assert min_value <= val <= max_value, f'Out of range: {colname} value of {val} is not between {min_value} and {max_value}'
        
print("all data tests passed successfully.")

# df.head(3)
df.dtypes

# Store prepped data in new CSV
Makes it easy to run in separate notebooks.

In [ ]:
outfile = os.path.splitext(infile)[0] + "_" + DATA_PREP_MODE + ".csv"
df.to_csv(outfile, index=False)
print(f"{DATA_PREP_MODE} data saved to new CSV file:\n{outfile}")